In [11]:
!pip install flask
!pip install pyserial
!pip install opencv-python
!pip install nbimporter
!pip install torch torchaudio torchvision
!jupyter nbconvert --to script text_processing.ipynb
!jupyter nbconvert --to script camera_operation.ipynb
!jupyter nbconvert --to script braille_translate.ipynb

^C
^C
^C
^C
^C
^C
^C


[NbConvertApp] Converting notebook braille_translate.ipynb to script
[NbConvertApp] Writing 6609 bytes to braille_translate.py


In [10]:
from flask import Flask, render_template, Response
import cv2

app = Flask(__name__)

# 웹캠에서 비디오 캡쳐
camera = cv2.VideoCapture(1)
camera.set(cv2.CAP_PROP_FRAME_WIDTH, 480)
camera.set(cv2.CAP_PROP_FRAME_HEIGHT, 270)

def generate_frames():
    while True:
        success, frame = camera.read()  # 카메라로부터 프레임 읽기
        if not success:
            break
        else:
            ret, buffer = cv2.imencode('.jpg', frame)
            frame = buffer.tobytes()
            yield (b'--frame\r\n'
                   b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')

@app.route('/')
def index():
    return render_template('braille.html')

@app.route('/video')
def video():
    return Response(generate_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__ == '__main__':
    app.run(debug=False, host='0.0.0.0', port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.0.18:5000
Press CTRL+C to quit
127.0.0.1 - - [01/Jun/2024 17:19:26] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Jun/2024 17:19:27] "GET /video HTTP/1.1" 200 -
127.0.0.1 - - [01/Jun/2024 17:19:53] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Jun/2024 17:19:53] "GET /video HTTP/1.1" 200 -


In [1]:
from flask import Flask, render_template, Response
import serial
import time
import os
import cv2
import threading
import subprocess
import nbimporter
from camera_operation import taking_picture
from image_processing import crop_image
from text_processing import text_correction, text_detection
from braille_translate import text_to_braille

app = Flask(__name__)

# 연결할 시리얼 포트와 baudrate 설정
ser = serial.Serial('COM7', 9600)

# 변수 선언 및 초기화
camera = cv2.VideoCapture(2)
recognized_texts_global = []
current_text_index = 0
braille_texts = []

def read_from_port(ser):
    # 인식된 텍스트를 저장할 전역 변수
    global recognized_texts_global
    global current_text_index
    global braille_texts

    while True:
        reading = ser.readline().decode().strip()  # 끝의 줄바꿈 제거
        print("===============  Received from Arduino -> ", reading, "  ===============")
        if reading == "Take Picture":
            # 이미지 촬영
            print("< 이미지 촬영 >")
            img_path = taking_picture(camera)

            print("< 텍스트 인식 >")
            recognized_texts = text_detection(img_path)
            # 인식된 텍스트 처리
            words_list = [word for text in recognized_texts for word in text.split()]
            corrected_texts = text_correction(words_list)
            filtered_texts = [text for text in corrected_texts if text.strip()]
            
            print("인식된 텍스트 : ", filtered_texts)
           # recognized_texts_global = filtered_texts  # 새로운 텍스트로 업데이트
           # current_text_index = 0  # 새로운 텍스트 리스트로 업데이트되면 인덱스 초기화

            
            # 텍스트를 점자로 변환하여 저장
            print("< 텍스트 점자로 변환 >")
            braille_texts = text_to_braille(filtered_texts)
            print("점자 텍스트 : ", braille_texts)
            recognized_texts_global = braille_texts
            current_text_index = 0
        elif reading == "Next":
            next_text()
            
        elif reading == "Previous":
            previous_text()
            
def next_text():
    global current_text_index
    
    if len(recognized_texts_global) == 0:
        print("리스트가 비어있습니다.")
        
    elif current_text_index < len(recognized_texts_global) - 1:
        send_text_to_arduino(recognized_texts_global[current_text_index])
        current_text_index += 1
    else:
        print("리스트의 끝점입니다.")

def previous_text():
    global current_text_index
    
    if len(recognized_texts_global) == 0:
        print("리스트가 비어있습니다.")
        
    elif current_text_index > 0:
        send_text_to_arduino(recognized_texts_global[current_text_index])
        current_text_index -= 1
        
    else:
        print("리스트의 시작점입니다.")

def send_text_to_arduino(text):
    print("Sending to Arduino:", text)
    ser.write((text + "\n").encode())

# 프로그램 종료 시 자원 해제
def release_resources():
    camera.release()
    ser.close()

@app.route('/')
def index():
    return render_template('braille.html')

@app.route('/video')
def video():
    return Response(generate_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

thread = threading.Thread(target=read_from_port, args=(ser,))
thread.start()

if __name__ == '__main__':
    app.run(debug=False, host='0.0.0.0', port=5000)

^C
^C


fatal: destination path 'yolov5' already exists and is not an empty directory.

KeyboardInterrupt



In [13]:
from flask import Flask, render_template, Response, jsonify
import serial
import time
import os
import cv2
import threading
import subprocess
import nbimporter
from camera_operation import taking_picture
from text_processing import text_correction, text_detection
from braille_translate import text_to_braille

app = Flask(__name__)

# 연결할 시리얼 포트와 baudrate 설정
ser = serial.Serial('COM7', 9600)

# 변수 선언 및 초기화
camera = cv2.VideoCapture(1)
recognized_texts_global = []
current_text_index = 0
braille_texts = []
log_messages = []
captured_image_path = None

def read_from_port(ser):
    global recognized_texts_global, current_text_index, braille_texts, log_messages, captured_image_path

    while True:
        reading = ser.readline().decode().strip()  # 끝의 줄바꿈 제거
        log_messages.append(f"Received from Arduino -> {reading}")
        if reading == "Take Picture":
            # 이미지 촬영
            log_messages.append("< 1 - 이미지 촬영 >")
            img_path = taking_picture(camera)
            captured_image_path = img_path

            # 널값인지 검사
            if img_path is None:
                continue

            # 텍스트 인식
            log_messages.append("< 2 - 텍스트 인식 >")
            recognized_texts = text_detection(img_path)
                
            # 인식된 텍스트 처리
            words_list = [word for text in recognized_texts for word in text.split()]
            try:
                corrected_texts = text_correction(words_list)
            except ValueError as e:
                log_messages.append(f"예외 발생: {e}")
                continue
            filtered_texts = [text for text in corrected_texts if text.strip()]
            
            log_messages.append(f"인식된 텍스트 >>> {filtered_texts}")
            recognized_texts_global = filtered_texts  # 새로운 텍스트로 업데이트
            current_text_index = 0  # 새로운 텍스트 리스트로 업데이트되면 인덱스 초기화

            
            # 텍스트를 점자로 변환하여 저장
            log_messages.append("< 3 - 텍스트 점자로 변환 >")
            braille_texts = text_to_braille(filtered_texts)
            log_messages.append(f"점자 텍스트 >>> {braille_texts}")
            
        elif reading == "Next":
            next_text()
            
        elif reading == "Previous":
            previous_text()
            
def next_text():
    global current_text_index, log_messages
    
    if len(recognized_texts_global) == 0:
        log_messages.append("[Warning] 리스트가 비어있습니다.")
        
    elif current_text_index < len(recognized_texts_global) - 1:
        send_text_to_arduino(recognized_texts_global[current_text_index])
        current_text_index += 1
        log_messages.append(f"Sending to Arduino: {recognized_texts_global[current_text_index]}")
    else:
        log_messages.append("리스트의 끝점입니다.")

def previous_text():
    global current_text_index, log_messages
    
    if len(recognized_texts_global) == 0:
        log_messages.append("[Warning] 리스트가 비어있습니다.")
        
    elif current_text_index > 0:
        send_text_to_arduino(recognized_texts_global[current_text_index])
        current_text_index -= 1
        log_messages.append(f"Sending to Arduino: {recognized_texts_global[current_text_index]}")
        
    else:
        log_messages.append("리스트의 시작점입니다.")

def send_text_to_arduino(text):
    ser.write((text + "\n").encode())

# 프로그램 종료 시 자원 해제
def release_resources():
    camera.release()
    ser.close()

def generate_frames():
    while True:
        success, frame = camera.read()  # 카메라로부터 프레임 읽기
        if not success:
            break
        else:
            ret, buffer = cv2.imencode('.jpg', frame)
            frame = buffer.tobytes()
            yield (b'--frame\r\n'
                   b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')

@app.route('/')
def index():
    return render_template('test1.html', captured_image_path=captured_image_path, log_messages=log_messages)

@app.route('/video')
def video():
    return Response(generate_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route('/labeled-image-url')
def labeled_image_url():
    # 외부 서버에 저장된 이미지의 URL
    external_image_url = ''
    # 외부 이미지 URL을 반환합니다.
    return external_image_url

@app.route('/logs')
def logs():
    return jsonify(log_messages)

thread = threading.Thread(target=read_from_port, args=(ser,))
thread.start()

if __name__ == '__main__':
    app.run(debug=False, host='0.0.0.0', port=5000)


AttributeError: module 'torch' has no attribute 'version'